In [9]:
# Multiple outputs per cell in Jupyter 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%load_ext jupyternotify
%autonotify -a 30

<IPython.core.display.Javascript object>

In [2]:
%%javascript
// Evitar autoscroll.
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [8]:
from integrado import *
%load_ext autoreload
%autoreload 2

In [14]:
tokens=Tokens().tokenize("data/geo-mx-2004_min.json",replace=True,showProgressEach=10000)

** Processing data/geo-mx-2004_min.json ...
2020-05-12 19:38:13.525968 :: 0.000227 - Processing item #0 
** Processed 10000 lines. Saved to /home/jovyan/public/B6_ProyectoIntegrador/data/geo-mx-2004_min.npy.


In [15]:
index_=Index().computePostingLists(tokens,showProgressEach=10000)
len(index_.postlists.keys())

** Counting TF & DF ...
2020-05-12 19:38:21.652881 :: 0.000827 - Processing item #0 
** Calculating TF-IDF...
2020-05-12 19:38:22.317649 :: 0.665595 - Processing item #0 
** Sorting posting lists...
2020-05-12 19:38:22.855857 :: 1.203803 - Processing item #0 
2020-05-12 19:38:22.955499 :: 1.303445 - Processing item #10000 


10383

In [17]:
ds=index_.o("covid coronavirus pandemia cuarentena",tokenize=True)
print(len(ds),ds[:5])
docs=tokens.getDocs(ds)
print(len(docs),docs[:5])
tokens.getTw

460 [11, 12, 39, 42, 104]
460 [['afortun', 'suel', 'sol', 'madr', 'cas', 'cuarenten', 'pas', 'si'], ['escuch', 'enlaz', 'inform', 'pandemi', 'retransmision', 'conciert', 'orquest', 'sinfon', 'nacional', 'honor', 'bethov', 'programacion'], ['altur', 'cuarenten', 'cuestion', 'si', 'amig', 'posit', 'pendej', 'comprob', 'toler'], ['si', 'sacrific', 'coloni', 'culiac', 'cov', 'seri', 'baranc'], ['dic', 'vay', 'acab', 'cov', 'da', 'cas', 'klosters']]


In [7]:
gc.collect()
import sys
local_vars = list(locals().items())
tot=0
vars_=[]
for var, obj in local_vars:
    mem=sys.getsizeof(obj)
    tot+=mem
    vars_.append((mem,var))
print("Total =",tot)
vars_.sort(reverse=True)
print(vars_)

20

Total = 21002
[(5688, 'ds'), (5504, 'docs'), (1064, 'Tokens'), (1064, 'InteractiveShell'), (1064, 'Index'), (1064, 'FastText'), (1064, 'Counter'), (400, 'defaultdict'), (278, '_i7'), (248, '_oh'), (248, 'Out'), (195, '_i1'), (172, '_i6'), (172, '_i'), (167, '_i2'), (144, 'tweet_iterator'), (138, '_iii'), (138, '_i4'), (136, '_ih'), (136, 'In'), (133, '_ii'), (133, '_i5'), (113, '__doc__'), (88, 'tb'), (88, 'sys'), (88, 'pd'), (88, 'os'), (88, 'np'), (88, 'gc'), (88, 'datetime'), (88, '__builtins__'), (88, '__builtin__'), (80, '_dh'), (72, 'get_ipython'), (72, '_i3'), (64, 'tokens'), (64, 'quit'), (64, 'index_'), (64, 'exit'), (57, '__name__'), (53, '___'), (53, '__'), (28, '_7'), (28, '_'), (16, '__spec__'), (16, '__package__'), (16, '__loader__')]
